<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3: Natural Language Processing Challenge (Part 3)


---

## Problem Statement


Our company is currently building a new wellness and mindfulness app to help promote better mental health. The app aims to provide thought of the day based on natural philosophies such as Buddhism and Stoicism. As a start, everyday, the app will allow the user to write one sentence against a prompt on certain topics. The app will then assess if the user is more in line with Buddhist or Stoic philosophies, and in return decide to return either Buddhist advice or Stoic advice based on the prompt written.

To train the app to gain an understanding of the types of people who will be more aligned with Buddhist or Stoic philosophies, we will be using posts from the subreddits r/Buddhism and r/Stoicism to machine train our model to distinguish and classify posts from the different subreddits.


# Content Page

---


# Datasets

- ['subreddit_posts_preprocessed'](./subreddit_posts_preprocessed.csv): Subreddit Posts Preprocessed


In part 2, we created a classification model by training several models and choosing the best accuracy score. In part 3, we will now try to extract the main discussion topics from each subreddit to help us better understand how to program our app. We can also gain into a more deeper insight into the commonalities and differences of users who frequent the two subreddits.


# Import Libraries


In [56]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


from nltk.corpus import stopwords

# Topic Modelling
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel


import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings('ignore')


In [57]:
# Set Pandas Options

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", None)

# Set Seaborn options
sns.set(style='ticks', context='talk')
plt.style.use('dark_background')


# Importing Data / Data Preprocessing


Since we have already preprocessed our data previously, we just import our preprocessed data.


In [58]:
# Import webscrapped data

subreddit_posts = pd.read_csv("subreddit_posts_preprocessed.csv")

subreddit_posts.shape


(19642, 5)

## Data Dictionary

| Feature    | Type  | Datasets(s)                  | Value(s)            | Description                                  |
| ---------- | ----- | ---------------------------- | ------------------- | -------------------------------------------- |
| subreddit  | _str_ | subreddit_posts_preprocessed | Buddhism / Stoicism | Subreddit classification of post             |
| title      | _str_ | subreddit_posts_preprocessed | -                   | Title of post from subreddit                 |
| selftext   | _str_ | subreddit_posts_preprocessed | -                   | Text Description of subreddit posts (if any) |
| clean_text | _str_ | subreddit_posts_preprocessed | -                   | Preprocessed title and selftext              |


We then split our data into two dataframes - one to hold the Buddhism subreddit posts, and the other to hold the Stoicism subreddit posts.


In [59]:
# Split the dataframe into two subreddits

df_buddhism = subreddit_posts.loc[subreddit_posts["subreddit"] == 0]
df_stoicism = subreddit_posts.loc[subreddit_posts["subreddit"] == 1]


# Topic Modelling


_Thanks to https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for the guide on topic modelling._

For topic modelling, we will be using GenSim to help us identify and extract the hidden topics in the various text. We will be using the Latent Dirichlet Allocation (LDA) algorithm to do so.

In LDA model, words are collected into documents and each word's presence is attributable to one of the document topics. The model automatically classifies any individual document within the collection in terms of how relevant it is to each of the discovered topics. A topic is considered to be a set of terms (i.e., individual words or phrases) that, taken together, suggest a shared theme.


We will be examining each subreddit to see the main topics.


## Buddhism


We will first create a function to tokenize our words and make it into a list for our model to process.


In [60]:
# Create a function to tokenize our words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


# Creating a text list for the text in our dataframe and tokenize them
text_list_buddhism = df_buddhism['clean_text'].tolist()
tokenized_posts_buddhism = list(sent_to_words(text_list_buddhism))


We will also be using GenSim to help us create bigrams in our text list that it has identified. We will then apply it to our text.


In [61]:
# Bigrams of text. Higher threshold fewer phrases.
bigram = gensim.models.Phrases(
    tokenized_posts_buddhism, min_count=5, threshold=80)

bigram_mod = gensim.models.phrases.Phraser(bigram)

# Define function for bigrams


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


# Applying the function to our tokenized posts
tokenized_posts_buddhism_bigrams = make_bigrams(tokenized_posts_buddhism)


We will need to create a dictionary and a corpus to fit into our model. Gensim creates a unique id for each word in the document. The produced corpus is a mapping of (word_id, word_frequency).


In [62]:
# Creating Buddhism Dictionary
dictionary_buddhism = Dictionary(tokenized_posts_buddhism_bigrams)

# Creating our Corpus
doc_term_matrix_buddhism = [dictionary_buddhism.doc2bow(
    post) for post in tokenized_posts_buddhism_bigrams]


Now we have to identify what number of topics will give us the best results for segregation. We will use the coherence score as our metric to judge how good our topic model is.

We will iterate through the number of topics to find out what is the number of topics that will give us the highest score.


In [63]:
# Function to iterate through number of topics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    LDA_model = gensim.models.ldamodel.LdaModel
    for num_topics in range(start, limit, step):
        model = LDA_model(corpus=corpus, num_topics=num_topics, id2word=dictionary,
                          random_state=42, chunksize=1000, passes=50, iterations=100)
        model_list.append(model)
        coherencemodel = CoherenceModel(
            model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


In [66]:
# Obtaining the coherence values into a list
model_list, coherence_values = compute_coherence_values(
    dictionary=dictionary_buddhism, corpus=doc_term_matrix_buddhism, texts=tokenized_posts_buddhism_bigrams, start=3, limit=19, step=3)

x = range(3, 19, 3)

# Zipping them together with the number of topics and print out the coherence values
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))


Num Topics = 3  has Coherence Value of 0.4037
Num Topics = 6  has Coherence Value of 0.3752
Num Topics = 9  has Coherence Value of 0.4664
Num Topics = 12  has Coherence Value of 0.4646
Num Topics = 15  has Coherence Value of 0.497
Num Topics = 18  has Coherence Value of 0.4812


Here we see that 15 topics give us the highest coherence value. We will be extracting the optimal model and dominant topic out from our model list and append the keywords and our text to a new dataframe as a side-by-side comparision.


In [67]:
# Extracting the model with 15 number of topics from our model list
optimal_model = model_list[4]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=10))


[(0, '0.018*"mind" + 0.018*"suffering" + 0.013*"self" + 0.012*"body" + 0.012*"experience" + 0.011*"thing" + 0.010*"thought" + 0.008*"desire" + 0.008*"make" + 0.007*"life"'), (1, '0.024*"meditation" + 0.022*"buddhism" + 0.021*"practice" + 0.016*"would" + 0.011*"like" + 0.011*"book" + 0.010*"know" + 0.010*"buddhist" + 0.009*"help" + 0.009*"ve"'), (2, '0.018*"people" + 0.017*"one" + 0.011*"would" + 0.011*"person" + 0.009*"time" + 0.008*"way" + 0.008*"say" + 0.007*"said" + 0.007*"others" + 0.007*"think"'), (3, '0.029*"dalai_lama" + 0.017*"youtube" + 0.015*"wanted_share" + 0.014*"passion" + 0.013*"meat" + 0.012*"valuable" + 0.012*"arahant" + 0.009*"similarity" + 0.009*"poison" + 0.009*"disease"'), (4, '0.074*"gt" + 0.054*"statue" + 0.020*"altar" + 0.017*"buddha" + 0.014*"bowl" + 0.013*"mala" + 0.011*"disrespectful" + 0.010*"picture" + 0.008*"interview" + 0.008*"sight"'), (5, '0.045*"retreat" + 0.035*"music" + 0.017*"observe" + 0.014*"rinpoche" + 0.011*"documentary" + 0.009*"bhante" + 0.008*

In [109]:
# Making it into a dataframe
sent_topics_df = pd.DataFrame()

# Get main topic in each document
for i, row in enumerate(optimal_model[doc_term_matrix_buddhism]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:  # => dominant topic
            wp = optimal_model.show_topic(topic_num)
            topic_keywords = ", ".join([word for word, prob in wp])
            sent_topics_df = sent_topics_df.append(pd.Series(
                [int(topic_num), round(prop_topic, 4), topic_keywords]), ignore_index=True)
        else:
            break

sent_topics_df.columns = ['Dominant_Topic',
                          'Perc_Contribution', 'Topic_Keywords']


In [111]:
# Number of Documents for Each Topic
topic_counts = sent_topics_df["Dominant_Topic"].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Concatenate Columns
df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)

df_dominant_topics.reset_index(inplace=True)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Num_Posts', 'Perc_Posts']

# Show
df_dominant_topics


,Dominant_Topic,Num_Posts,Perc_Posts
0,1,1898,0.1935
1,2,1648,0.1680
2,6,1587,0.1618
3,11,1490,0.1519
4,0,1076,0.1097
5,10,635,0.0647
6,14,450,0.0459
7,8,434,0.0443
8,9,229,0.0234
9,4,75,0.0076


We see that the top 5 categories form about ~75% of the posts in our dataset. These are Dominant Topic 1, 2, 6, 11, and 0. We can individually try to sieve out the posts that the algorithm has identified to contributed the highest to the respective categories to identify the main topics of discussion.


### Interpreting the topic keywords


In [112]:
# Creating a dataframe to hold all the topics

dominant_topics_buddhism = pd.DataFrame(
    columns=["Dominant_Topic", "Perc_Contribution", "Topic_Keywords"])

for i in range(0, 16):

    dominant_topics_buddhism = pd.concat([dominant_topics_buddhism, sent_topics_df.loc[(
        sent_topics_df["Dominant_Topic"] == i)].sort_values("Perc_Contribution", ascending=False).head(5)], sort=False)


In [113]:
# Extracting the selftext from the original dataframe

df_buddhism_index = dominant_topics_buddhism.index.tolist()

df_index = df_buddhism.iloc[df_buddhism_index]

dominant_topics_buddhism = dominant_topics_buddhism.join(df_index)


## Dominant Topic 0


In [114]:
dominant_topics_buddhism.loc[dominant_topics_buddhism["Dominant_Topic"] == 0]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,subreddit,title,selftext,clean_text,message_length
1776,0,0.9844,"mind, suffering, self, body, experience, thing, thought, desire, make, life",0,"¤¤¤ Weekly /r/Buddhism General Discussion ¤¤¤ - May 02, 2022 - New to Buddhism? Read this first!","This thread is for general discussion, such as brief thoughts, notes, updates, comments, or questions that don't require a full post of their own. Posts here can include topics that are discouraged on this sub in the interest of maintaining focus, such as sharing meditative experiences, drug experiences related to insights, discussion on dietary choices for Buddhists, and others. Conversation will be much more loosely moderated than usual, and generally only frankly unacceptable posts will be removed.\n\nIf you are new to Buddhism, you may want to start with our [FAQs](https://www.reddit.com/r/Buddhism/wiki/faq) and have a look at the other resources in the [wiki](https://www.reddit.com/r/Buddhism/wiki/index). If you still have questions or want to hear from others, feel free to post here or make a new post.\n\nYou can also use this thread to dedicate the merit of our practice to others and to make specific aspirations or prayers for others' well-being.",¤¤¤ weekly buddhism general discussion ¤¤¤ may new buddhism read first thread general discussion brief thought note update comment question dont require full post post include topic discouraged sub interest maintaining focus sharing meditative experience drug experience related insight discussion dietary choice buddhist others conversation much loosely moderated usual generally frankly unacceptable post removedif new buddhism may want start look resource still question want hear others feel free post make new postyou also use thread dedicate merit practice others make specific aspiration prayer others wellbeing,85
117,0,0.9844,"mind, suffering, self, body, experience, thing, thought, desire, make, life",0,"¤¤¤ Weekly /r/Buddhism General Discussion ¤¤¤ - May 30, 2022 - New to Buddhism? Read this first!","This thread is for general discussion, such as brief thoughts, notes, updates, comments, or questions that don't require a full post of their own. Posts here can include topics that are discouraged on this sub in the interest of maintaining focus, such as sharing meditative experiences, drug experiences related to insights, discussion on dietary choices for Buddhists, and others. Conversation will be much more loosely moderated than usual, and generally only frankly unacceptable posts will be removed.\n\nIf you are new to Buddhism, you may want to start with our [FAQs](https://www.reddit.com/r/Buddhism/wiki/faq) and have a look at the other resources in the [wiki](https://www.reddit.com/r/Buddhism/wiki/index). If you still have questions or want to hear from others, feel free to post here or make a new post.\n\nYou can also use this thread to dedicate the merit of our practice to others and to make specific aspirations or prayers for others' well-being.",¤¤¤ weekly buddhism general discussion ¤¤¤ may new buddhism read first thread general discussion brief thought note update comment question dont require full post post include topic discouraged sub interest maintaining focus sharing meditative experience drug experience related insight discussion dietary choice buddhist others conversation much loosely moderated usual generally frankly unacceptable post removedif new buddhism may want start look resource still question want hear others feel free post make new postyou also use thread dedicate merit practice others make specific aspiration prayer others wellbeing,85
1387,0,0.9844,"mind, suffering, self, body, experience, thing, thought, desire, make, life",0,"¤¤¤ Weekly /r/Buddhism General Discussion ¤¤¤ - May 09, 2022 - New to Buddhism? Read this first!","This thread is for general discussion, such as brief thoughts, notes, updates, comments, or questions that don't require a full post of t

Dominant Topic 0 focuses mainly on the general discussion thread on every subreddit.


## Dominant Topic 1


In [115]:
dominant_topics_buddhism.loc[dominant_topics_buddhism["Dominant_Topic"] == 1]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,subreddit,title,selftext,clean_text,message_length
8506,1,0.9611,"meditation, buddhism, practice, would, like, book, know, buddhist, help, ve",0,Beginner Buddhist and Taoist readings?,Hello I’ve recently become interested in learning about Buddhism and Taoism but I don’t entirely know where to start. Do you guys have any beginner Buddhist or Taoist books to help me start to understand the topic better?,beginner buddhist taoist reading hello i’ve recently become interested learning buddhism taoism don’t entirely know start guy beginner buddhist taoist book help start understand topic better,26
9521,1,0.9576,"meditation, buddhism, practice, would, like, book, know, buddhist, help, ve",0,New to this sub and want to learn more.,I saw something one day that made me more curious about Buddhism. \n\nI'm looking to achieve more balance...understand the root of Buddhism and hopefully come to a path that helps enlighten me more in life.,new sub want learn saw something one day made curious buddhism looking achieve balanceunderstand root buddhism hopefully come path help enlighten life,22
3682,1,0.9533,"meditation, buddhism, practice, would, like, book, know, buddhist, help, ve",0,"Hi, where do I start if I want to study buddhism?","Hi, I find things that I have read or heard about buddhism interesting and I like its philosophy, I would like to learn more, where do I start? Please advise.\n\nBest,",hi start want study buddhism hi find thing read heard buddhism interesting like philosophy would like learn start please advisebest,20
7510,1,0.9481,"meditation, buddhism, practice, would, like, book, know, buddhist, help, ve",0,So how do you all go about cultivating concentration in your meditation?,It’s one of the many many things I’m building on in my practice. I’m making progress but was thinking I could get some perspectives on here.,go cultivating concentration meditation it’s one many many thing i’m building practice i’m making progress thinking could get perspective,19
9480,1,0.9282,"meditation, buddhism, practice, would, like, book, know, buddhist, help, ve",0,"I'm very interested in Buddhism, but so far I only know the basic tenets. What books or resources can you recommend to expand my knowledge base?",NaN,interested buddhism far know basic tenet book resource recommend expand knowledge base,12


Dominant Topic 1 seems to be posts that are about seeking deeper knowledge about basic buddhism philosophy and practices. It seems many posts from this category are new users to Buddhism and are posting in the subreddit to seek for help on how to live a Buddhist life with Buddhist habits.


## Dominant Topic 2


In [116]:
dominant_topics_buddhism.loc[dominant_topics_buddhism["Dominant_Topic"] == 2]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,subreddit,title,selftext,clean_text,message_length
763,2,0.9777,"people, one, would, person, time, way, say, said, others, think",0,Would a monk eat a carrot?,"A carrot is the taproot, and harvesting it requires the plant's early death. To be edible, carrots need to be harvested before they get too woody, which is before the plant's natural death.\n\nDoes eating a carrot go against the teaching of ""do not kill""? Are we only to eat those plant parts that are readily offered by the plant for the purpose of eating (and dispersing seeds) without killing it (e.g. apples), or that otherwise do not cause death (e.g. leaves)?",would monk eat carrot carrot taproot harvesting requires plant early death edible carrot need harvested get woody plant natural deathdoes eating carrot go teaching kill eat plant part readily offered plant purpose eating dispersing seed without killing eg apple otherwise cause death eg leaf,44
8132,2,0.9576,"people, one, would, person, time, way, say, said, others, think",0,"I believe I am the most important person in the world for a good reason, because of this technique that I found which will change the world. Until I have shared it enough, I will remain the most important person until the weight of the world is off my shoulders, so please take it off and share.",NaN,believe important person world good reason technique found change world shared enough remain important person weight world shoulder please take share,21
613,2,0.9533,"people, one, would, person, time, way, say, said, others, think",0,"Regular Day-to-Day Words Have Power To Cast Spells On Yourself and Others!? 🗣⚡ Your participation is your line of safety in difficult times 🛡 and it's easy as Giving a Like 💚, Commenting 💯, Subscribing and sharing with your Friends",NaN,regular daytoday word power cast spell others participation line safety difficult time easy giving like commenting subscribing sharing friend,19
4212,2,0.9354,"people, one, would, person, time, way, say, said, others, think",0,Right speech question,"“Right Speech: no lying, no rude speech, no telling one person what another says about him to cause discord or harm their relationship”\n\nSo my question is what if someone you know is cheating on their wife and think the right thing to do is tell her, however, telling her means not following right speech, right? And if you tell her, it would 99% likely harm their relationship.\n\nWhat would you do in this situation?",right speech question right speech lying rude speech telling one person another say cause discord harm relationshipso question someone know cheating wife think right thing tell however telling mean following right speech right tell would likely harm relationshipwhat would situation,40
8142,2,0.9333,"people, one, would, person, time, way, say, said, others, think",0,Which precept is the hardest to live by?,Could someone tell me which precept is the hardest to live by with a reason why it is? I would love to know!,precept hardest live could someone tell precept hardest live reason would love know,13


Dominant Topic 2 seems to be posts that deal with moral dilemmas and philosophical questions in Buddhism. It seems that users are posting in the subreddit to seek others perspectives and advice on the philosophical questions and moral dilemmas in Buddhism philosophies.


## Dominant Topic 6


In [117]:
dominant_topics_buddhism.loc[dominant_topics_buddhism["Dominant_Topic"] == 6]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,subreddit,title,selftext,clean_text,message_length
7578,6,0.9481,"buddhism, buddhist, buddha, religion, teaching, would, believe, one, question, people",0,Can a Buddhist follow a path where they believe in God?,"Can a Buddhist choose to follow a path that believes in a concept of God?\n\nNot in a Jewish, Christian, Muslim, or anything like that\n\nBut a view of God not connected to a Religion.",buddhist follow path believe god buddhist choose follow path belief concept godnot jewish christian muslim anything like thatbut view god connected religion,22
7641,6,0.9451,"buddhism, buddhist, buddha, religion, teaching, would, believe, one, question, people",0,"Can I be Buddhist without belief in supernatural beings or gods? I have heard this question from others but the answer is always “belief isn’t the issue, disbelief is the problem” but what exactly does that mean?",NaN,buddhist without belief supernatural god heard question others answer always belief isn’t issue disbelief problem exactly mean,17
589,6,0.9417,"buddhism, buddhist, buddha, religion, teaching, would, believe, one, question, people",0,What's the point of following one particular school of Buddhism if technically all of the teachngs still come from Buddha? Wouldn't it be better to follow a mix of all?,NaN,whats point following one particular school buddhism technically teachngs still come buddha wouldnt better follow mix,16
5187,6,0.9333,"buddhism, buddhist, buddha, religion, teaching, would, believe, one, question, people",0,A discourse about Buddhism and Jainism.,"What is the difference between Jainism and Buddhism? (I'm new to both these terms, also i understand categorisation is not something one should majorly be concerned about, hence I'm just curious)",discourse buddhism jainism difference jainism buddhism new term also understand categorisation something one majorly concerned hence curious,17
7210,6,0.9282,"buddhism, buddhist, buddha, religion, teaching, would, believe, one, question, people",0,Do Buddhists have to worship the Buddha?,Do Buddhists have to worship the Buddha to be a proper Buddhist? And what does it mean exactly to worship the Buddha?,buddhist worship buddha buddhist worship buddha proper buddhist mean exactly worship buddha,12


Dominant Topic 6 seems to be about posts discussing comparisons between other religions and buddhism and the belief and worship of gods.


## Dominant Topic 11


In [119]:
dominant_topics_buddhism.loc[dominant_topics_buddhism["Dominant_Topic"] == 11]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,subreddit,title,selftext,clean_text,message_length
5242,11,0.9699,"like, feel, dont, life, know, get, thing, time, want, really",0,EVERYONE NEEDS TO HELP EVERYONE,EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE..........EVERYONE NEEDS TO HELP EVERYONE.,everyone need help everyone everyone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyoneeveryone need help everyone,41
7339,11,0.9481,"like, feel, dont, life, know, get, thing, time, want, really",0,How I feel sometimes when I start to meditate...,Anyone else feel like this sometimes? :D\n\nLet me know if you have any similar ideas to this you would like to see and I could try and make them.,feel sometimes start meditate anyone else feel like sometimes dlet know similar idea would like see could try make,19
735,11,0.9221,"like, feel, dont, life, know, get, thing, time, want, really",0,Why is it so hard to sit still and do nothing?,"I can maybe do it for a few minutes but not more than that.\n\nBTW, I'd prefer concise replies if possible.",hard sit still nothing maybe minute thatbtw id prefer concise reply possible,12
639,11,0.9105,"like, feel, dont, life, know, get, thing, time, want, really",0,Therapy still not here,"Starting to think my mom lied about therapy, don't believe nothing my fam say nomore after all the years of abuse. My mom told me a month ago I would be seeing a therapist in person and my mental health clinic where I get my meds from haven't said nothing yet either, this session was booked a few months ago. They only text my mother now and not me. Feeling lonely and I nobody to talk to and my grandmother who used to be my best friend suddenly start gaslighting me starting 2 or so years ago. Ever since I've been diagnosed with bipolar disorder people stop reaching out as soon as I got better and started taking my meds like I should and right around the time I try to get my own money, health insurance and do things on my own my family doesn't want to help, they just get my hopes up only for them to lie again and again. I post in bipolar, and psychiatry sub reddit but it's so small nobody even replies so i reach out to yall.",therapy still starting think mom lied therapy dont believe nothing fam say nomore year abuse mom told month ago would seeing therapist person mental health clinic get med havent said nothing yet either session booked month ago text mother feeling lonely nobody talk grandmother used best friend suddenly start gaslighting starting year ago ever since diagnosed bipolar disorder people stop reaching soon got better started taking med like right around time try get money health insurance thing family doesnt want help get hope lie post bipolar psychiatry sub reddit small nobody even reply reach yall,95
841,11,0.9096,"like, feel, dont, life, know, get, thing, time, want, really",0,I just can't build habits,"I always try to build good habits but I fail. I wake up very late. Sleep very late. Miss breakfast. I joined the gym but went only twice. I do the bare minimum to get done in my job.\n\nI tried to build a practice of meditation but that ain't working either. \n\nI just feel like if we all die one day what's the point even. I just spend all day scrolling Instagram or YouTube.\n\nI am software engineer, so I have to practice Programming, but I can't get myself to. I feel like my job i

Dominant Topic number 11 seems to be about users finding help in cultivating long-term buddhist habits such as rituals, meditation, and therapy. It seems that users are posting for advice on problems that they face in their daily lives.


## Stoicisim


We replicate what we have done with the posts from the Buddhism Subreddit with the Stoicism Subreddit.


In [120]:
# Creating a text list for the text in our dataframe and tokenize them
text_list_stoicism = df_stoicism['clean_text'].tolist()
tokenized_posts_stoicism = list(sent_to_words(text_list_stoicism))

# Bigrams of text. Higher threshold fewer phrases.
bigram = gensim.models.Phrases(
    tokenized_posts_stoicism, min_count=5, threshold=80)

bigram_mod = gensim.models.phrases.Phraser(bigram)

# Define function for bigrams


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


# Applying the function to our tokenized posts
tokenized_posts_stoicism_bigrams = make_bigrams(tokenized_posts_stoicism)

# Creating Stoicism Dictionary
dictionary_stoicism = Dictionary(tokenized_posts_stoicism_bigrams)

# Creating our Corpus
doc_term_matrix_stoicism = [dictionary_stoicism.doc2bow(
    post) for post in tokenized_posts_stoicism_bigrams]

# Obtaining the coherence values into a list
model_list, coherence_values = compute_coherence_values(
    dictionary=dictionary_stoicism, corpus=doc_term_matrix_stoicism, texts=tokenized_posts_stoicism_bigrams, start=3, limit=19, step=3)

x = range(3, 19, 3)

# Zipping them together with the number of topics and print out the coherence values
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))


Num Topics = 3  has Coherence Value of 0.3478
Num Topics = 6  has Coherence Value of 0.4178
Num Topics = 9  has Coherence Value of 0.4911
Num Topics = 12  has Coherence Value of 0.5321
Num Topics = 15  has Coherence Value of 0.5113
Num Topics = 18  has Coherence Value of 0.5158


It seems that the best number of topics is 12 based on the coherence score. We shall use that for our model assessment.


In [121]:
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=10))


[(0, '0.016*"like" + 0.014*"feel" + 0.012*"get" + 0.011*"know" + 0.011*"time" + 0.009*"people" + 0.009*"want" + 0.009*"dont" + 0.009*"would" + 0.008*"year"'), (1, '0.017*"podcast" + 0.012*"encourage" + 0.009*"black" + 0.008*"necessity" + 0.008*"jesus" + 0.007*"victory" + 0.007*"donald_robertson" + 0.006*"plato" + 0.006*"humor" + 0.006*"audiobook"'), (2, '0.058*"god" + 0.019*"religious" + 0.014*"religion" + 0.012*"rain" + 0.012*"contrary" + 0.011*"hence" + 0.008*"style" + 0.008*"tattoo" + 0.007*"existed" + 0.006*"faith"'), (3, '0.034*"dog" + 0.011*"book_recommendation" + 0.009*"relief" + 0.008*"closed" + 0.008*"spoken" + 0.007*"tackle" + 0.007*"programming" + 0.007*"cleaning" + 0.006*"irritated" + 0.006*"musonius_rufus"'), (4, '0.012*"famous" + 0.012*"journaling" + 0.011*"psychology" + 0.010*"essay" + 0.008*"seneca_letter" + 0.006*"attributed" + 0.006*"similarity" + 0.005*"manual" + 0.005*"rep" + 0.004*"practical_application"'), (5, '0.059*"stoicism" + 0.050*"stoic" + 0.020*"book" + 0.0

In [154]:
# Making it into a dataframe
sent_topics_df = pd.DataFrame()

# Get main topic in each document
for i, row in enumerate(optimal_model[doc_term_matrix_stoicism]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:  # => dominant topic
            wp = optimal_model.show_topic(topic_num)
            topic_keywords = ", ".join([word for word, prob in wp])
            sent_topics_df = sent_topics_df.append(pd.Series(
                [int(topic_num), round(prop_topic, 4), topic_keywords]), ignore_index=True)
        else:
            break

sent_topics_df.columns = ['Dominant_Topic',
                          'Perc_Contribution', 'Topic_Keywords']


In [159]:
# Number of Documents for Each Topic
topic_counts = sent_topics_df["Dominant_Topic"].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Concatenate Columns
df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)

df_dominant_topics.reset_index(inplace=True)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Num_Posts', 'Perc_Posts']

# Show
df_dominant_topics


,Dominant_Topic,Num_Posts,Perc_Posts
0,0,4614,0.4691
1,9,2890,0.2938
2,5,1987,0.2020
3,8,232,0.0236
4,10,25,0.0025
5,6,24,0.0024
6,11,18,0.0018
7,3,12,0.0012
8,7,11,0.0011
9,1,10,0.0010


We see that compared to the Buddhism Subreddit, the number of dominant topics seems to be fewer as the top 3 topics occupy ~ 95% of the entire subreddit. The top 3 topics are Dominant Topic 0, 9, 5.


In [160]:
# Creating a dataframe to hold all the topics
dominant_topics_stoicism = pd.DataFrame(
    columns=["Dominant_Topic", "Perc_Contribution", "Topic_Keywords"])

for i in range(0, 13):

    dominant_topics_stoicism = pd.concat([dominant_topics_stoicism, sent_topics_df.loc[(
        sent_topics_df["Dominant_Topic"] == i)].sort_values("Perc_Contribution", ascending=False).head(5)], sort=False)


In [169]:
df_stoicism_index = dominant_topics_stoicism.index.tolist()

df_stoicism.reset_index(inplace=True)

df_index = df_stoicism.iloc[df_stoicism_index]

dominant_topics_stoicism = dominant_topics_stoicism.join(df_index)


## Dominant Topic 0


In [170]:
dominant_topics_stoicism.loc[dominant_topics_stoicism["Dominant_Topic"] == 0]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,level_0,index,subreddit,title,selftext,clean_text,message_length
126,0,0.992,"like, feel, get, know, time, people, want, dont, would, year",126,9933,1,How to move on,"So this February i met a girl with whom i felt a connection i've never felt before. I was ready to pursue her romantically, but after 3 weeks she told me she was a lesbian. I was super down but i decided to stick around because i thought 'ok she is not into me, and she never will, but I just like her so much as a person , i think we could be friends or something else' \n\nThis was a dumb decision because even though i told myself that i just wanted to be her friend, i really was just hoping for something to happen. The more i got to know her the more i fell in love with her. I became more nervous around her and my mind started to be flooded by thoughts of her to the point of starting to get desperate. \n\nAfter putting some distance, i see that what i was doing was wrong and honestly i feel like an idiot. One of the reasons i liked her so much is because it has been so long since i actually met someone i genuinely like that i disregarded every thing that was telling me to stop thinking about her in that light. However, even after all of this we are still close and i fell like a really want to be there as her friend, without hoping for anything. It is pretty damn hard but i think i really want to give it a shot.\n\nI want to move on and refocus on my own self growth, but i'm really straggling to do so. What stoic principles can i use to refocus on myself and start moving forward? What should i tell myself?",move february met girl felt connection never felt ready pursue romantically week told lesbian super decided stick around thought ok never like much person think could friend something else dumb decision even though told wanted friend really hoping something happen got know fell love became nervous around mind started flooded thought point starting get desperate putting distance see wrong honestly feel like idiot one reason liked much long since actually met someone genuinely like disregarded every thing telling stop thinking light however even still close fell like really want friend without hoping anything pretty damn hard think really want give shoti want move refocus self growth really straggling stoic principle use refocus start moving forward tell,116
6819,0,0.9914,"like, feel, get, know, time, people, want, dont, would, year",6819,16626,1,How do you deal when someone you love keeps on doing something infuriating?,"How do you, as a stoic, deal when someone you love keeps on doing something that deeply upsets you?\n\nMy SO has depression and anxiety and there are good weeks and then there are bad weeks. SO refused to let me know what was actually bothering her/him to the point she/he made a request to not talk for 1 week and that just infuriated me.\n\nHer/His reason for not telling me this is because I have a very high stress job and he/she didn't want to add on to my ""stress."" It's not because I complain about my job (I voice out my problems from time to time) that makes her/him think it's high stress. It's because I work for a public official.\n\nI'm really patient when it comes to him/her but keeping her/his issue a secret for weeks until it blew up in my face just made me extremely upset and made me say some things that hurt. Her/His request wouldn't have infuriated me if he/she would would have voiced out the issues first.\n\nWe had fights like this before wherein it blew up in my face but this is the first time I said hurtful things. I try to apply stoicism to every aspect of my life but her/his request just made me forget stoicism.\n\nI would appreciate any feedback on your experiences in dealing with similar issues.\n\nJust to be clear: WHAT INFURIATED ME WAS NOT BECAUSE SHE/HE HAS DEPRESSION ANXIETY IT WAS BECAUSE HE/SHE CHOSE TO KEEP THE BAD WEEKS A SECRET FROM ME UNTIL IT BLEW UP IN MY FACE.",de

Dominant Topic 0 seems to be posts about users who are facing relationship problems or major lifestyle problems. Many are posts seeking advice from other users on how to overcome these problems with stoicism principles.


## Dominant Topic 5


In [172]:
dominant_topics_stoicism.loc[dominant_topics_stoicism["Dominant_Topic"] == 5]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,level_0,index,subreddit,title,selftext,clean_text,message_length
2203,5,0.9863,"stoicism, stoic, book, philosophy, read, reading, would, meditation, like, quote",2203,12010,1,"READ BEFORE POSTING: r/Stoicism beginner's guide, weekly discussion thread, FAQ, and rules","Welcome to the r/Stoicism subreddit, a forum for discussion of Stoicism, the school of philosophy founded by Zeno of Citium in the 3rd century BC. Please use the comments of this post for beginner's questions and general discussion.\n\n&amp;nbsp; \n\n# r/Stoicism Beginner's Guide\n\n- **Please read [our FAQ](http://www.reddit.com/r/Stoicism/wiki/guide).**\n- **Review the [subreddit rules](https://www.reddit.com/r/stoicism/about/rules/) and [Reddiquette](https://reddit.zendesk.com/hc/en-us/articles/205926439-Reddiquette).**\n- Check out [frequently discussed topics](https://www.reddit.com/r/Stoicism/wiki/guide#wiki_frequently_discussed_topics).\n- Get familiar with Stoic theory and practice from excellent [community contributions](https://www.reddit.com/r/stoicism/wiki/communitycontentselections).\n- See [additional wiki contents](https://www.reddit.com/r/Stoicism/about/wiki/contents).\n\n# External Stoicism Resources\n- The Internet Encyclopedia of Philosophy's general [entry on Stoicism](https://www.iep.utm.edu/stoicism/).\n- The Stanford Encyclopedia of Philosophy's more technical [entry on Stoicism](https://plato.stanford.edu/entries/stoicism/). \n- The Routledge Encyclopedia of Philosophy's thorough [entry on Stoicism](https://www.rep.routledge.com/articles/thematic/stoicism/v-1).\n- For an abbreviated, basic, and non-technical introduction, see [here](https://donaldrobertson.name/2013/02/03/a-simplified-modern-approach-to-stoicism/) and [here](https://donaldrobertson.name/2013/02/20/introduction-to-stoicism-the-three-disciplines/). \n\n# Stoic Texts in the Public Domain\n\n- Please visit the subreddit [Library](https://www.reddit.com/r/Stoicism/about/wiki/library) for freely available Stoic texts.",read posting stoicism beginner guide weekly discussion thread faq rule welcome stoicism subreddit forum discussion stoicism school philosophy founded zeno citium century bc please use comment post beginner question general discussionampnbsp stoicism beginner guide please read review check get familiar stoic theory practice excellent see external stoicism resource internet encyclopedia philosophy general stanford encyclopedia philosophy technical routledge encyclopedia philosophy thorough abbreviated basic nontechnical introduction see stoic text public domain please visit subreddit freely available stoic text,76
40,5,0.9861,"stoicism, stoic, book, philosophy, read, reading, would, meditation, like, quote",40,9847,1,"READ BEFORE POSTING: r/Stoicism beginner's guide, weekly discussion thread, FAQ, and rules","Welcome to the r/Stoicism subreddit, a forum for discussion of Stoicism, the school of philosophy founded by Zeno of Citium in the 3rd century BC. Please use the comments of this post for beginner's questions and general discussion.\n\n&amp;nbsp; \n\n# r/Stoicism Beginner's Guide\n\n- **Please read [our FAQ](http://www.reddit.com/r/Stoicism/wiki/guide).**\n- **Review the [subreddit rules](https://www.reddit.com/r/stoicism/about/rules/) and [Reddiquette](https://reddit.zendesk.com/hc/en-us/articles/205926439-Reddiquette).**\n- Check out [frequently discussed topics](https://www.reddit.com/r/Stoicism/wiki/guide#wiki_frequently_discussed_topics).\n- Get familiar with Stoic theory and practice from excellent [community contributions](https://www.reddit.com/r/stoicism/wiki/communitycontentselections).\n- See [additional wiki contents](https://www.reddit.com/r/Stoicism/wiki/contents).\n\n# External Stoicism Resources\n- The Internet Encyclopedia of Philosophy's general [entry on Stoicism](https://www.iep.utm.edu/stoicism/).\n- The Stanford Encyclopedia of Philosophy's more technical [entry on Stoicism](https://plato.st

Dominant Topic 5 seems to be posts from new users who want to learn deeper about stoicism principles and are seeking advice on resources about Stoicism philosophies.


## Dominant Topic 9


In [173]:
dominant_topics_stoicism.loc[dominant_topics_stoicism["Dominant_Topic"] == 9]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,level_0,index,subreddit,title,selftext,clean_text,message_length
872,9,0.9916,"life, thing, one, way, control, make, time, mind, thought, think",872,10679,1,"Other people are not 'wrong' simply because they failed to be what we expected or hoped for them to be. If people are not what we'd hoped, we were wrong.","If you pick up a puzzle piece and it doesn't fit were you thought, it's not a bad piece. It was a bad guess on our part. It's us, always 100% us.\n\nOther people are never wrong. Of course we try to improve things going forward but the present is exactly what it is.\n\nWe're not ever really wrong, either. The fault may be in our own judgement, but it is natural and normal for us to find our way as best we can through uncertainty. Our best guess is always the right play. We are responsible in the practical sense but we are no more to blame than anyone else. \n\nI'm not suggesting that we withhold judgement of others and ourselves; judgement is fine,. I'm saying that blame is meaningless here, it's like blaming the moon for being round, it's ridiculous, conceptually incoherent. Others simply are what they are, despite our imaginings. We simply are what we are, despite our imaginings. \n\nOf course we desire to improve, and our noble expectations of ourselves are our guides going forward, they are not the present truth. If they are well-selected guides, we'd rightly expect to sometimes fall short and be corrected by them. It is admirable to aim high, admirable to be corrected and to adapt ourselves to it. In fact, this is arguably the only meaningful measure of progress.",people wrong simply failed expected hoped people wed hoped wrong pick puzzle piece doesnt fit thought bad piece bad guess part always usother people never wrong course try improve thing going forward present exactly iswere ever really wrong either fault may judgement natural normal find way best uncertainty best guess always right play responsible practical sense blame anyone else suggesting withhold judgement others judgement fine saying blame meaningless like blaming moon round ridiculous conceptually incoherent others simply despite imaginings simply despite imaginings course desire improve noble expectation guide going forward present truth wellselected guide wed rightly expect sometimes fall short corrected admirable aim high admirable corrected adapt fact arguably meaningful measure progress,112
895,9,0.9859,"life, thing, one, way, control, make, time, mind, thought, think",895,10702,1,"""I ask not for a lighter burden, but for broader shoulders.""","When Zeus punished Atlas(the embodiment of endurance) for waging war against the gods, he placed the world on his back. Whilst carrying the weight of the world, Atlas(supposedly) pleaded to Zeus: ""I ask not for a lighter burden, but for broader shoulders."" The quote has also been cited as a Jewish proverb, so the true origin is foggy.\n\nIn my opinion the origin doesn't completely matter; the essence remains the same. In life the weight of your duties and struggles is typically out of your control, therefore there is only one element that you can control to fix an overbearing burden; it is you. However lets say you *could* change the weight. Would it not still be better to rise above the challenge?\n\nThe quote embodies this.",ask lighter burden broader shoulder zeus punished atlasthe embodiment endurance waging war god placed world back whilst carrying weight world atlassupposedly pleaded zeus ask lighter burden broader shoulder quote also cited jewish proverb true origin foggyin opinion origin doesnt completely matter essence remains life weight duty struggle typically control therefore one element control fix overbearing burden however let say could change weight would still better rise challengethe quote embodies,69
8547,9,0.9854,"life, thing, one, way, control, make, time, mind, thought, think",8547,18354,1,Feeling disconnected from life,"I know s

Dominant Topic 9 seems to be about posts with random quotes, examples and allegories, with the purpose to enlighten others. There is a focus on controlling one's thoughts and focusing on positive thinking to help overcome life's challenges.


---

# Summary and Conclusion


## Topic Modelling Business Insights


<b>Similarities</b>

1. Both subreddits have people who are seeking clarifications on the different philosophies tenets. One insight we could glean is that Buddhism and Stoic users are more intellectually curious in nature and could be a target demographic in our marketing efforts.

2. Users of the subreddits both gravitate to Buddhism or Stoicism to seek help and other user's perspectives on problems. This is a useful insight as it shows a potential target audience for our mental health and wellness app as users seek Buddhist or Stoicism advice to overcome life's challenges.

<b> Differences </b>

1. Buddhism reddit users seems to question more about religion and their philosophical tenets more than Stoicism reddit users. This could show that people who gravitate to Buddhism are more questioning in nature compared to Stoic users. We could use this as a key distinguishing factor to determine the type of prompts returned.

2. Buddhism reddit users gravitate towards meditation and rituals, while stoicism reddit users are focused more on positive thinking, and control. This could be a distinguishing factor when looking at keywords. So when a user's prompt leans towards positive thinking and control, the app will provide stoicism prompts while if a user references religion, it can return Buddhism prompts instead.

3. Buddhism reddit users also seem to be less verbose compared to Stoicism reddit users looking at the message length and context of the post. Hence, message length could also be a distinguishing feature.

4. Buddhism reddit users seems to be more outward looking and would think about things in a bigger perspective. In contrast, the posts from stoicism seems to have more introspective look at life with a focus on one's own circumstances.

## Limitations to the project

1. The data is only being scraped from reddit who may not represent the entire view of Buddhism or Stoicism. Hence, data might be biased.

2. The modelling parameters could be better tuned if there were better processing power and time available.

3. Model accuracies are within standard deviation of each other. So it is hard to determine at this point which is better. We will probably need to better test the accuracy of our model once our app is beta tested.
